In [ ]:
import json

from sklearn.model_selection import train_test_split

In [ ]:
!pip3 install -U scikit-learn

## A script to format regulat export to one that can be used for ultralistic yolo training

In [ ]:
label_file = 'data/labels.json'

labels = {}
with open(label_file, 'r') as f:
    labels = json.load(f)
    labels = labels['labels']
labels

In [ ]:
ids = set(map(lambda x: x['series_id'], labels))
ids = list(ids)
len(ids)

In [ ]:
ratio = 0.2
train_ids, val_ids = train_test_split(ids, test_size=ratio)
len(train_ids), len(val_ids)

In [ ]:
from PIL import Image

fn = labels[0]['image_path']
i = Image.open(fn)
width, height = i.size
width, height

In [ ]:
import os
import shutil

from tqdm.notebook import tqdm

classs_mappig = {
    'anchor': 0,
    'dart': 1
}

def build_label(box_size, pos_x, pos_y, img_width, img_height):
    """
    https://docs.ultralytics.com/datasets/detect/#ultralytics-yolo-format
    """
    center_x, center_y = pos_x / img_width , pos_y / img_height
    width, height = box_size / img_width, box_size / img_height
    return center_x, center_y, width, height

def get_fn(path):
    return path.split('/')[-1]

def copy_img(label, out_dir):
    img_path = label['image_path']
    
    out_path = out_dir + '/' + get_fn(img_path)

    # Create missing directories if they don't exist
    destination_directory = os.path.dirname(out_path)
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)
        
    shutil.copy(img_path, out_path)

def build_label_file(label, out_dir, img_size, box_size=50):
    img_fn = get_fn(label['image_path'])
    label_fn = '.'.join(img_fn.split('.')[0:-1] ) + '.txt'
    out_path = out_dir + '/' + label_fn

    img_width, img_height = img_size
    annotations = []
    for a in label['annotations']:
        c = classs_mappig[a['label']]
        pos_x, pos_y = a['pos']['x'], a['pos']['y']
        left, top, width, height = build_label(box_size, pos_x, pos_y, img_width, img_height)
        label_str = f'{c} {left} {top} {width} {height}'
        annotations.append(label_str)
    with open(out_path, 'w') as f:
        content = '\n'.join(annotations)
        f.write(content)
        
out_dir_base = 'data_yolo'
out_dir_train = out_dir_base + '/train'
out_dir_val = out_dir_base + '/val'


for label in tqdm(labels):
    series_id = label['series_id']
    is_train = not series_id in val_ids
    out_dir = out_dir_train if is_train else out_dir_val

    copy_img(label, out_dir)
    build_label_file(label, out_dir, (width, height), box_size=70)